In [1]:
import numpy as np
import os

In [2]:
base_script = """#!/bin/bash

#SBATCH --job-name={}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16GB
#SBATCH --gres=gpu:1
#SBATCH --time=100:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --output="o{}-%j.out"

module purge
module load anaconda3/4.3.1
module load sox/intel/14.4.2
module load ffmpeg/intel/3.2.2
module load cuda/10.1.105
module load cudnn/10.0v7.6.2.24

source activate py3.5

SLICES='{}'
MODELNAME='{}'
TRAINFEATURENAMES="{}"
FULLFEATURENAMES="{}"

cd /home/ci411/pcen-t-varying/scripts/core_scripts

python 02_train.py --slices=$SLICES --model-id=$MODELNAME --model-dir='/beegfs/ci411/pcen/models/models_122' --feature-dir='/beegfs/ci411/pcen/features/' --feature-names=$TRAINFEATURENAMES --load-pump='/beegfs/ci411/pcen/pumps/full_test'
python 03_evaluate.py --slices=$SLICES --model-id=$MODELNAME --model-dir='/beegfs/ci411/pcen/models/models_122' --feature-dir='/beegfs/ci411/pcen/features/' --feature-names=$FULLFEATURENAMES"""

In [3]:
#{name, name, slices, modelname, trainfeatures, fullfeatures}

In [4]:
file_out = 'test.sbatch'
train_features = ['URBAN-SED_dry']
full_features = ['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long','URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel']
full_features_str = str(full_features).replace(' ','')
with open(file_out, 'w+') as f:
    f.write(base_script.format('test', 'test', '[0,3,5]', 'test_model', train_features, full_features_str))

In [6]:
#slice variations
slice_variations = [list(np.arange(n+1)) for n in range(10)]

slice_variations_2 = [list(np.arange(n, 10-n)) for n in range(4,0,-1)]

slices_all_variations = slice_variations+slice_variations_2
print(slices_all_variations)

[[0], [0, 1], [0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [4, 5], [3, 4, 5, 6], [2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8]]


In [7]:
slices_all_variations = [[i] for i in range(10)]
print(slices_all_variations)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]


In [8]:
#make dry scripts
train_features = ['URBAN-SED_dry']
full_features = ['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long','URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel']
full_features_str = str(full_features).replace(' ','')
for slices in slices_all_variations:
    slice_str = ''.join(map(str, slices))
    name = 'dry'+slice_str
    modelname = "model122_dry_" + slice_str
    filename = modelname + '.sbatch'
    slice_list_str = str(slices).replace(' ','')
    sbatch_str = base_script.format(name,name,slice_list_str,modelname,train_features,full_features_str)
    with open(filename, 'w+') as f:
        f.write(sbatch_str)

In [9]:
#make wet scripts
for slices in slices_all_variations:
    slice_str = ''.join(map(str, slices))
    name = 'wet'+slice_str
    modelname = "model122_wet_" + slice_str
    filename = modelname + '.sbatch'
    slice_list_str = str(slices).replace(' ','')
    #make training/testing features the full set
    sbatch_str = base_script.format(name,name,slice_list_str,modelname,full_features_str,full_features_str)
    with open(filename, 'w+') as f:
        f.write(sbatch_str)

In [10]:
os.listdir()
models = [item for item in os.listdir() if "model122" in item and '.sbatch' in item]

In [11]:
models.sort()
models

['model122_dry_0.sbatch',
 'model122_dry_01.sbatch',
 'model122_dry_012.sbatch',
 'model122_dry_0123.sbatch',
 'model122_dry_01234.sbatch',
 'model122_dry_012345.sbatch',
 'model122_dry_0123456.sbatch',
 'model122_dry_01234567.sbatch',
 'model122_dry_012345678.sbatch',
 'model122_dry_0123456789.sbatch',
 'model122_dry_1.sbatch',
 'model122_dry_12345678.sbatch',
 'model122_dry_2.sbatch',
 'model122_dry_234567.sbatch',
 'model122_dry_3.sbatch',
 'model122_dry_3456.sbatch',
 'model122_dry_4.sbatch',
 'model122_dry_45.sbatch',
 'model122_dry_5.sbatch',
 'model122_dry_6.sbatch',
 'model122_dry_7.sbatch',
 'model122_dry_8.sbatch',
 'model122_dry_9.sbatch',
 'model122_wet_0.sbatch',
 'model122_wet_01.sbatch',
 'model122_wet_012.sbatch',
 'model122_wet_0123.sbatch',
 'model122_wet_01234.sbatch',
 'model122_wet_012345.sbatch',
 'model122_wet_0123456.sbatch',
 'model122_wet_01234567.sbatch',
 'model122_wet_012345678.sbatch',
 'model122_wet_0123456789.sbatch',
 'model122_wet_1.sbatch',
 'model122

In [77]:
with open('runall.bat', 'w+') as f:
    for model in models:
        f.write('sbatch '+ model + '\n')
        

In [12]:
with open('runsingle.bat', 'w+') as f:
    for i in range(10):
        for term in ['wet','dry']:
            f.write('sbatch model122_' + term + '_' + str(i) +'.sbatch\n')